# TP2 : Image classification

* 1 Bag-of-Words Classication with Histograms Oriented Gradients

In [ ]:
import cv2
import numpy
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter, convolve
import scipy.ndimage as ndimage
import pandas as pd

In [ ]:
def grid(image, n_points_x, n_points_y):
    
    # Get the shape of the image
    height, width = image.shape
    
    # Compute the 1D vectors 
    X_vect = numpy.linspace(0, width-1, n_points_x)
    Y_vect = numpy.linspace(0, height-1, n_points_y)
    
    # Creating the meshgrid
    X, Y = numpy.meshgrid(X_vect, Y_vect)

    # Flatens the matrix to get a vector with cordinated
    X_flat = X.ravel()
    Y_flat = Y.ravel()

    return X_flat, Y_flat, X, Y

# Exemple d'utilisation
image = cv2.imread('STL10/stl10_raw/test/airplane/airplane_test_001.png')

# Convertir l'image en niveaux de gris (important pour que la fonction grid fonctionne correctement)
image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

n_points_x = 6 # Nombre de points dans la direction x
n_points_y = 6  # Nombre de points dans la direction y
# Appel de la fonction
X_flat, Y_flat, X, Y = grid(image_gray, n_points_x, n_points_y)

# Affichage des premiers résultats pour visualiser la grille
print("Indices X :", X_flat[:10])
print("Indices Y :", Y_flat[:10])

plt.scatter(X, Y, c='r', marker='o')  # Utilisation de scatter pour visualiser la grille de points
plt.gca().invert_yaxis()  # Inverser l'axe Y pour correspondre aux coordonnées d'image
plt.title('Visualisation du meshgrid')
plt.xlabel('X')
plt.ylabel('Y')
plt.show()

In [ ]:
def compute_hog_descriptors(image, X_flat, Y_flat, cell_size=4, num_bins=8, patch_size=16):
    """
    param image: image (grayscale)
    :param X_flat: 1D vector of x coordinates
    :param Y_flat: 1D vector of y coordinates
    :param cell_size: cell size (par défaut 4x4 pixels)
    :param num_bins: number of bins of each patch (8 by default)
    :return: 2D array containing hog descriptors for each point of the grid
    """
    hog_descriptors = []
    height, width = image.shape

    # Comoute the image gradient (using Sobel)
    gx = ndimage.sobel(image, axis=1)  # Gradient in x
    gy = ndimage.sobel(image, axis=0)  # Gradient in y

    # Compute the magnitude and direction of the gradient
    magnitude = np.sqrt(gx**2 + gy**2)
    direction = np.arctan2(gy, gx)  

    # Put it in angle form
    direction = np.mod(direction, 2 * np.pi)


    # Pour chaque point de la grille, calculer le descripteur HOG
    for x, y in zip(X_flat, Y_flat):
        x, y = int(x), int(y)

        # Calculer à quel patch le point appartient
        x_patch_min = (x // patch_size) * patch_size
        y_patch_min = (y // patch_size) * patch_size

        # Fixer les limites du patch
        x_min = x_patch_min
        x_max = min(x_min + patch_size, width)
        y_min = y_patch_min
        y_max = min(y_min + patch_size, height)

        # Extraire la magnitude et la direction des gradients pour le patch de 16x16 pixels
        patch_magnitude = magnitude[y_min:y_max, x_min:x_max]
        patch_direction = direction[y_min:y_max, x_min:x_max]

        # Divide the patch into a 4x4 cells
        hog_vector = []
        
        # For each cell we compute the histogram
        for i in range(0, patch_magnitude.shape[0], cell_size):
            for j in range(0, patch_magnitude.shape[1], cell_size):
                # Extract the 4x4 cell from the patch
                cell_magnitude = patch_magnitude[i:i+cell_size, j:j+cell_size]
                cell_direction = patch_direction[i:i+cell_size, j:j+cell_size]

                # Create a histogram for that cell
                hist, _ = np.histogram(
                    cell_direction, bins=num_bins, range=(0, 2 * np.pi),
                    weights=cell_magnitude
                )

                # Append every histogram for each cell to the vector
                hog_vector.extend(hist)

        # Apend the histogram of this patch to the list of hog descriptor
        hog_descriptors.append(hog_vector)

    # Return the array of the HOG descriptor
    return np.array(hog_descriptors)

HOG_D = compute_hog_descriptors(image_gray, X_flat, Y_flat)



In [ ]:
def display_hog_matrix(hog_descriptors):
    """
    Show informations of the hog descriptors
    
    :param hog_descriptors: 2D matrix of the hog descriptor (n_points x 128)
    """
    print("Shape of the HOG descriptors matrix:", hog_descriptors.shape)
    print("\nFirst 5 rows of the HOG descriptors matrix:\n", hog_descriptors[:5])  # Afficher les 5 premières lignes

# Example for an image
hog_descriptors = compute_hog_descriptors(image_gray, X_flat, Y_flat)
display_hog_matrix(hog_descriptors)



In [ ]:
import os
import cv2

def load_images_from_folders(root_folder):
    """
    Loads all grayscale images from subfolders of a root folder.
    
    :param root_folder: Path to the root folder containing class subfolders.
    :return: List of images and corresponding labels (class indices).
    """
    images = []
    labels = []
    class_folders = sorted(os.listdir(root_folder))  # Sort to ensure consistent class order

    for class_index, folder in enumerate(class_folders):
        folder_path = os.path.join(root_folder, folder)
        if os.path.isdir(folder_path):
            for filename in os.listdir(folder_path):
                    image_path = os.path.join(folder_path, filename)
                    # Load the image in grayscale
                    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                    if image is not None:
                        images.append(image)
                        labels.append(class_index)  # Assign the class label based on folder index
    return images, labels

# Example usage
root_folder_train = 'STL10/stl10_raw/train'  # Path to the root folder containing class subfolders
root_folder_test = 'STL10/stl10_raw/test'
train_images, train_labels = load_images_from_folders(root_folder_train)
test_images, test_labels = load_images_from_folders(root_folder_test)

print(len(test_labels))


In [ ]:
def compute_hog_for_all_images(images, X_flat, Y_flat):
    """
    Computes HOG descriptors for all images in the dataset and stores them in a single list.
    
    :param images: List of grayscale images.
    :param X_flat, Y_flat: Grid points for HOG descriptor extraction.
    :return: A list containing the HOG descriptors for each image and a combined matrix of all HOG descriptors.
    """
    all_hog_descriptors = []  # To store HOG descriptors for all images
    combined_hog_matrix = []  # To store all HOG descriptors in one matrix (for K-means, etc.)

    for i, image in enumerate(images):
        # Compute HOG descriptors for the current image
        hog_descriptors = compute_hog_descriptors(image, X_flat, Y_flat)

        # Append the HOG descriptors of this image to the list
        all_hog_descriptors.append(hog_descriptors)

        # Combine HOG descriptors into a single matrix for all images
        combined_hog_matrix.append(hog_descriptors)

    # Convert the combined matrix to a 2D NumPy array
    combined_hog_matrix = np.vstack(combined_hog_matrix)

    return all_hog_descriptors, combined_hog_matrix



In [ ]:
import numpy as np
from scipy.spatial.distance import cdist

def kmeans_clustering(hog_descriptors, K, max_iters=100, tol=1e-4):
    """
    Implements the K-means clustering algorithm to build a visual vocabulary.
    
    :param hog_descriptors: 2D array (n_samples x 128) containing HOG descriptors from all images
    :param K: Number of clusters (size of the visual vocabulary)
    :param max_iters: Maximum number of iterations for K-means (default 100)
    :param tol: Tolerance to stop the algorithm when centroids converge (default 1e-4)
    :return: 2D array of size K x 128 containing the final cluster centroids (visual words)
    """
    # Step 1: Randomly initialize the cluster centroids
    n_samples, n_features = hog_descriptors.shape
    print(hog_descriptors.shape)
    centroids = hog_descriptors[np.random.permutation(n_samples)[:K]]  # Randomly select K centroids from data

    for iteration in range(max_iters):
        # Step 2: Assign each descriptor to the nearest centroid (using Euclidean distance)
        distances = cdist(hog_descriptors, centroids, 'euclidean')
        labels = np.argmin(distances, axis=1)  # Find the nearest centroid for each descriptor

        # Save the current centroids to check convergence later
        previous_centroids = centroids.copy()

        # Step 3: Update centroids by calculating the mean of all descriptors assigned to each centroid
        for k in range(K):
            points_in_cluster = hog_descriptors[labels == k]
            if len(points_in_cluster) > 0:
                centroids[k] = points_in_cluster.mean(axis=0)  # New centroid is the mean of all assigned points

        # Check for convergence (if centroids don't change much)
        centroid_shift = np.linalg.norm(centroids - previous_centroids, axis=1).max()
        if centroid_shift < tol:
            print(f"K-means converged after {iteration+1} iterations.")
            break

    return centroids

# Example usage with sample HOG descriptors
# Assume hog_descriptors is a 2D array (n_samples x 128) containing all HOG descriptors from training images


In [ ]:
# X_flat and Y_flat are the grid coordinates for extracting HOG descriptors for both train et test set
all_hog_descriptors_train, combined_hog_matrix_train = compute_hog_for_all_images(train_images, X_flat, Y_flat)
all_hog_descriptors_test, combined_hog_matrix_test = compute_hog_for_all_images(test_images, X_flat, Y_flat)

#Perform K-means clustering on the combined HOG descriptors matrix
K = 50  # Number of clusters (size of the visual vocabulary)
vocabulary = kmeans_clustering(combined_hog_matrix_train, K)

#Visualize the resulting vocabulary shape
print("Visual vocabulary (centroids):", vocabulary.shape)


In [97]:
def compute_bow_histograms(hog_descriptors_list, vocabulary, labels):
    """
    Computes the BoW histograms for all images in the dataset.
    
    :param hog_descriptors_list: List of 2D arrays, where each array contains the HOG descriptors of an image
    :param vocabulary: 2D array (K x 128) of visual words (centroids from K-means)
    :param labels: List or array containing the ground truth labels for each image
    :return: 2D array of shape (n_images, K) containing the BoW histograms, and 1D array of ground truth labels
    """
    K = vocabulary.shape[0]  # Number of clusters (visual words)
    n_images = len(hog_descriptors_list)

    # Initialize the BoW histograms and ground truth labels arrays
    bow_histograms = np.zeros((n_images, K))
    ground_truth_labels = np.array(labels)

    # For each image, compute the BoW histogram
    for i, hog_descriptors in enumerate(hog_descriptors_list):
        # Compute distances between each HOG descriptor and the vocabulary (centroids)
        distances = cdist(hog_descriptors, vocabulary, 'euclidean')

        # Find the nearest centroid (visual word) for each descriptor
        nearest_centroids = np.argmin(distances, axis=1)

        # Count the occurrences of each visual word using np.bincount
        bow_histogram = np.bincount(nearest_centroids, minlength=K)

        # Normalize the histogram to get a distribution (optional but recommended)
        bow_histogram = bow_histogram / np.sum(bow_histogram)

        # Store the BoW histogram for this image
        bow_histograms[i, :] = bow_histogram

    return bow_histograms, ground_truth_labels

# Example usage
# Assume hog_descriptors_list is a list where each element is a 2D array (n_patches x 128) of HOG descriptors for an image
# Assume vocabulary is the array of centroids from K-means
# Assume labels is the list of ground truth labels for each image

bow_histograms_train, ground_truth_labels_train = compute_bow_histograms(all_hog_descriptors_train, vocabulary, train_labels)
bow_histograms_test, ground_truth_labels_test = compute_bow_histograms(all_hog_descriptors_test, vocabulary, test_labels)

# Print the shape of the resulting BoW histograms and ground truth labels
print(f"BoW histograms shape: {bow_histograms_train.shape}")
print(f"Ground truth labels shape: {ground_truth_labels_test.shape}")



BoW histograms shape: (5000, 100)
Ground truth labels shape: (8000,)
8000


In [101]:
print(len(bow_histograms_test))
print(len(bow_histograms_train))

print(len(all_hog_descriptors_train))
print(len(all_hog_descriptors_test))
print(len(test_labels))

8000
5000
5000
8000
8000


In [69]:
import numpy as np
from scipy.spatial.distance import cdist

def knn_classifier(train_bow_histograms, train_labels, test_bow_histograms, k=3):
    """
    Implements a k-Nearest Neighbors (k-NN) classifier using Euclidean (L2) distance.
    
    :param train_bow_histograms: 2D array (n_train_samples x K) containing the BoW histograms of the training images
    :param train_labels: 1D array (n_train_samples) containing the labels of the training images
    :param test_bow_histograms: 2D array (n_test_samples x K) containing the BoW histograms of the test images
    :param k: The number of nearest neighbors to consider (default is 3)
    :return: 1D array (n_test_samples) containing the predicted labels for the test images
    """
    #convert trainlables to a numpy
    train_labels = np.array(train_labels)
    
    # Step 1: Compute the distances between each test BoW histogram and all train BoW histograms
    distances = cdist(test_bow_histograms, train_bow_histograms, 'euclidean')

    # Step 2: Find the indices of the k nearest neighbors for each test image
    nearest_neighbors = np.argsort(distances, axis=1)[:, :k]

    # Step 3: Predict the label of each test image based on the majority vote of its k nearest neighbors
    predicted_labels = []
    for neighbors in nearest_neighbors:
        # Get the labels of the k nearest neighbors
        neighbor_labels = train_labels[neighbors]

        # Find the most common label (majority vote)
        unique_labels, counts = np.unique(neighbor_labels, return_counts=True)
        majority_label = unique_labels[np.argmax(counts)]

        # Append the predicted label
        predicted_labels.append(majority_label)

    return np.array(predicted_labels)

# Example usage:
# train_bow_histograms: BoW histograms of the training images (shape: n_train_samples x K)
# train_labels: Labels of the training images (shape: n_train_samples)
# test_bow_histograms: BoW histograms of the test images (shape: n_test_samples x K)

k = 3  # Number of neighbors to consider
predicted_labels = knn_classifier(bow_histograms_train, train_labels, bow_histograms_test, k)

# Print predicted labels for the test set
print("Predicted labels for test images:", predicted_labels)

Predicted labels for test images: [8 7 7 ... 0 2 5]


# Experiments

In [94]:
def filter_airplane_bird(hog_descriptors_list, labels):
    """
    Filter the HOG descriptors to keep only the "airplane" (0) and "bird" (1) classes.
    
    :param hog_descriptors_list: List of HOG descriptors (one 2D array per image)
    :param labels: Array of corresponding labels for the dataset.
    :return: Filtered HOG descriptors and labels containing only airplane and bird classes.
    """
    filtered_descriptors = []
    filtered_labels = []
    for descriptors, label in zip(hog_descriptors_list, labels):
        if label == 0 or label == 1:  # Keep only airplane (0) and bird (1)
            filtered_descriptors.append(descriptors)
            filtered_labels.append(label)
    return np.array(filtered_descriptors), np.array(filtered_labels)

# Filter the training and test sets to keep only airplane and bird classes
filtered_train_descriptors, filtered_train_labels = filter_airplane_bird(all_hog_descriptors_train, train_labels)
filtered_test_descriptors, filtered_test_labels = filter_airplane_bird(all_hog_descriptors_test, test_labels)

In [88]:
# Apply K-means clustering on the HOG descriptors from the training set (airplane and bird only)
vocabulary = kmeans_clustering(combined_hog_matrix_train, K=100, max_iters=10)

(180000, 128)


In [106]:
# Calculate BoW histograms for the training and test sets using the vocabulary
bow_histograms_train, ground_truth_labels_train = compute_bow_histograms(filtered_train_descriptors, vocabulary, filtered_train_labels)
bow_histograms_test, ground_truth_labels_test = compute_bow_histograms(filtered_test_descriptors, vocabulary, filtered_test_labels)

# Print the length of the BoW histograms to check they match the number of images
print(f"Number of BoW histograms in training set: {len(bow_histograms_train)}")  # Should match the number of training images (e.g., 1000)
print(f"Number of BoW histograms in test set: {len(bow_histograms_test)}")  # Should match the number of test images (e.g., 1600)

Number of BoW histograms in training set: 1000
Number of BoW histograms in test set: 1600


In [110]:
from sklearn.metrics import accuracy_score

# Use k-NN to predict the labels for the test images
predicted_labels = knn_classifier(bow_histograms_train, filtered_train_labels, bow_histograms_test, k=3)

# Print accuracy for the "airplane" and "bird" classes
accuracy = accuracy_score(filtered_test_labels, predicted_labels)
print(f"Accuracy for airplane and bird: {accuracy * 100:.2f}%")


Computing distances...
Distances computed. Shape: (1600, 1000)
Finding nearest neighbors...
Nearest neighbors shape: (1600, 3)
Test image 0 neighbors' labels: [0 0 0]
Test image 1 neighbors' labels: [0 1 1]
Test image 2 neighbors' labels: [1 0 1]
Test image 3 neighbors' labels: [0 1 0]
Test image 4 neighbors' labels: [0 0 0]
Test image 5 neighbors' labels: [1 0 1]
Test image 6 neighbors' labels: [1 0 0]
Test image 7 neighbors' labels: [1 1 0]
Test image 8 neighbors' labels: [1 1 1]
Test image 9 neighbors' labels: [1 0 0]
Test image 10 neighbors' labels: [0 1 1]
Test image 11 neighbors' labels: [0 0 0]
Test image 12 neighbors' labels: [0 0 0]
Test image 13 neighbors' labels: [0 0 1]
Test image 14 neighbors' labels: [0 1 1]
Test image 15 neighbors' labels: [1 1 0]
Test image 16 neighbors' labels: [0 1 1]
Test image 17 neighbors' labels: [0 1 0]
Test image 18 neighbors' labels: [1 0 0]
Test image 19 neighbors' labels: [1 1 1]
Test image 20 neighbors' labels: [0 0 0]
Test image 21 neighbor

In [114]:
n_runs = 10  # Number of runs for training and evaluation
accuracies = []  # Store accuracy results for each run

for run in range(n_runs):
    # Re-train K-means for each run
    vocabulary = kmeans_clustering(combined_hog_matrix_train, K=100, max_iters=10)

    # Recompute the BoW histograms with the new vocabulary
    bow_histograms_train, ground_truth_labels_train  = compute_bow_histograms(filtered_train_descriptors, vocabulary, filtered_train_labels)
    bow_histograms_test, ground_truth_labels_test = compute_bow_histograms(filtered_test_descriptors, vocabulary, filtered_test_labels)

    # Predict the labels for the test set using k-NN
    predicted_labels = knn_classifier(bow_histograms_train, filtered_train_labels, bow_histograms_test, k=3)

    # Compute and store the accuracy for this run
    accuracy = accuracy_score(filtered_test_labels, predicted_labels)
    accuracies.append(accuracy)
    print(f"Run {run + 1}: Accuracy = {accuracy * 100:.2f}%")

# Calculate and print the mean accuracy over all 10 runs
mean_accuracy = np.mean(accuracies)
print(f"Mean accuracy over 10 runs: {mean_accuracy * 100:.2f}%")

(180000, 128)
Computing distances...
Distances computed. Shape: (1600, 1000)
Finding nearest neighbors...
Nearest neighbors shape: (1600, 3)
Test image 0 neighbors' labels: [0 0 0]
Test image 1 neighbors' labels: [1 1 0]
Test image 2 neighbors' labels: [1 1 1]
Test image 3 neighbors' labels: [0 1 1]
Test image 4 neighbors' labels: [0 1 0]
Test image 5 neighbors' labels: [0 0 0]
Test image 6 neighbors' labels: [0 0 0]
Test image 7 neighbors' labels: [1 1 0]
Test image 8 neighbors' labels: [1 1 1]
Test image 9 neighbors' labels: [1 0 0]
Test image 10 neighbors' labels: [0 0 1]
Test image 11 neighbors' labels: [0 0 0]
Test image 12 neighbors' labels: [0 0 0]
Test image 13 neighbors' labels: [0 0 0]
Test image 14 neighbors' labels: [1 1 0]
Test image 15 neighbors' labels: [0 1 1]
Test image 16 neighbors' labels: [1 1 1]
Test image 17 neighbors' labels: [1 1 0]
Test image 18 neighbors' labels: [1 0 0]
Test image 19 neighbors' labels: [1 0 1]
Test image 20 neighbors' labels: [0 0 0]
Test ima